In [1]:
import numpy as np
import time
import requests
import math
import itertools
from bs4 import BeautifulSoup
from fractions import Fraction
from IPython.display import Markdown, display

In [2]:
url='https://www.janestreet.com/puzzles/bracketology-101-index/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
#display([(i,j) for i,j in enumerate(y)])
display(Markdown("### "+y[7]+"\n\n"+str("\n".join(y[9:12]))))

### Bracketology 101

Show Solution
There’s a certain insanity in the air this time of the year that gets us thinking about tournament brackets.  Consider a tournament with 16 competitors, seeded 1-16, and arranged in the single-elimination bracket pictured above (identical to a “region” of the NCAA Division 1 basketball tournament). Assume that when the X-seed plays the Y-seed, the X-seed has a Y/(X+Y) probability of winning. E.g. in the first round, the 5-seed has a 12/17 chance of beating the 12-seed.
Suppose the 2-seed has the chance to secretly swap two teams’ placements in the bracket before the tournament begins. So, for example, say they choose to swap the 8- and 16-seeds. Then the 8-seed would play their first game against the 1-seed and have a 1/9 chance of advancing to the next round, and the 16-seed would play their first game against the 9-seed and have a 9/25 chance of advancing.

### Puzzle details
<img src="https://www.janestreet.com/puzzles/2021-04-01-bracketology-101.png" width="250">

In [3]:
# function to calculate the probability of seed winning for a given bracket
def calc(seed,bracket):
    n = len(bracket)
    # work out the odds of each match. 
    odds = [[Fraction(j,i+j) for j in bracket] for i in bracket]
    
    round = list(np.ones(n,dtype=int))
    
    # loop through the rounds mutiplying out the probabilities
    for r in range(int(math.log(n,2))):
        
        #for each team work out the set of possible opponents in a given round
        opps = [i for i in range(int(2**r),0,-1)]+[-i for i in range(int(2**r),2**(r+1))]
        
        # multiply the probability of team i being in the round by 
        # the sum of the products of 
        # - the chances of each of the possible opponenets making it through and 
        # - the chance of team i winning against that opponenet
        round = [round[i]*sum([round[i+opps[i % 2**(r+1)]+j]*odds[i][i+opps[i % 2**(r+1)]+j] for j in range(2**r)]) for i in range(n)]

    return round[bracket.index(seed)]

In [4]:
seed = 2
bracket = [1,16,8,9,5,12,4,13,6,11,3,14,7,10,2,15]

display(Markdown("For the original bracket :"))
display(Markdown("Probability of seed {} winning is {:.2f}%".format(seed,float(calc(seed,bracket))*100)))

swap = (3,16)
bracket2 = bracket.copy()
bracket2[bracket.index(swap[0])],bracket2[bracket.index(swap[1])] = bracket2[bracket.index(swap[1])],bracket2[bracket.index(swap[0])]

display(Markdown("\nSwapping {}-{} gives :".format(swap[0],swap[1])))
display(Markdown("Probability of seed {} winning is {:.2f}%".format(seed,float(calc(seed,bracket2))*100)))
display(Markdown("\n**Difference is {:.5f}%**".format(float(calc(seed,bracket2)-calc(seed,bracket))*100)))

For the original bracket :

Probability of seed 2 winning is 21.60%


Swapping 3-16 gives :

Probability of seed 2 winning is 28.16%


**Difference is 6.55795%**

In [5]:
print("\nRunning all possibilities")
print("=========================")
diffs =[]
print("Swap\tProb {} wins\t Diff".format(seed))
print("----\t-----------\t ----".format(seed))
for swap in [i for i in itertools.combinations([*range(1,len(bracket)+1)],2)]:
    bracket2 = bracket.copy()
    bracket2[bracket.index(swap[0])],bracket2[bracket.index(swap[1])] = bracket2[bracket.index(swap[1])],bracket2[bracket.index(swap[0])]
    print("{}-{}".format(swap[0],swap[1]),end="\t")
    print("{:7.2f}%".format(float(calc(seed,bracket2))*100),end="\t")
    print("{:+5.6f}%".format(float(calc(seed,bracket2)-calc(seed,bracket))*100))
    diffs.append(calc(seed,bracket2)-calc(seed,bracket))
display(Markdown("`Best Swap is : "+str([i for i in itertools.combinations([*range(1,len(bracket)+1)],2)][np.argmax(diffs)])+"`"))


Running all possibilities
Swap	Prob 2 wins	 Diff
----	-----------	 ----
1-2	  23.03%	+1.424294%
1-3	  19.92%	-1.683503%
1-4	  22.08%	+0.471132%
1-5	  22.37%	+0.770788%
1-6	  21.85%	+0.250925%
1-7	  16.88%	-4.726948%
1-8	  22.04%	+0.436367%
1-9	  22.09%	+0.488198%
1-10	  17.45%	-4.153344%
1-11	  22.99%	+1.386153%
1-12	  23.43%	+1.830058%
1-13	  23.67%	+2.062520%
1-14	  23.97%	+2.365184%
1-15	  13.58%	-8.021029%
1-16	  21.60%	+0.000000%
2-3	  21.07%	-0.529686%
2-4	  18.40%	-3.199723%
2-5	  17.53%	-4.077609%
2-6	  19.43%	-2.169239%
2-7	  20.67%	-0.935237%
2-8	  15.28%	-6.322926%
2-9	  14.96%	-6.640242%
2-10	  20.29%	-1.316813%
2-11	  18.01%	-3.592429%
2-12	  14.92%	-6.686996%
2-13	  14.75%	-6.852023%
2-14	  17.52%	-4.086014%
2-15	  21.60%	+0.000000%
2-16	  13.78%	-7.823535%
3-4	  22.91%	+1.309133%
3-5	  23.81%	+2.210638%
3-6	  21.72%	+0.116452%
3-7	  20.49%	-1.112796%
3-8	  26.03%	+4.428705%
3-9	  26.23%	+4.628463%
3-10	  20.92%	-0.684985%
3-11	  22.12%	+0.516822%
3-12	  25.55%	+3.950247

`Best Swap is : (3, 16)`

In [6]:
url='https://www.janestreet.com/puzzles/bracketology-101-solution/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
#display([(i,j) for i,j in enumerate(y)])
display(Markdown("### "+y[8]+"\n\n"+str("\n".join(y[10:16]))))

### April 2021 : Solution

The most straightforward way to solve this puzzle was to compute the
probability distributions of the winners of each match recursively,
given each swap.  The most advantageous swap for the 2-seed is to swap
seeds
3 and 16
, which increases the 2-seed’s probability of winning by
6.55795%
.  One easy mistake to make was to accidentally report the
1-seed’s probability of winning after swapping the 2-seed with the
1-seed.  This swap is good for the 2-seed, but only increases their
probability of winning from 21.6040% to 23.0283%, so 1.4243%.
The following puzzlers managed to find the correct swap and the
increase in probability.

In [7]:
# less generic and elegant (but more readable) code

def calc(seed,bracket):
    n = len(bracket)
    choice0 = [ 1,-1]
    choice1 = [ 2, 1,-2,-3]
    choice2 = [ 4, 3, 2, 1,-4,-5,-6,-7]
    choice3 = [ 8, 7 ,6, 5, 4, 3, 2, 1,-8,-9,-10,-11,-12,-13,-14,-15]

    odds = [[Fraction(j,i+j) for j in bracket] for i in bracket]
    round0 = list(np.ones(n))
    
    round1 = [round0[i]*sum([odds[i][i+choice0[(i % 2)]+j] for j in range(1)]) for i in range(n)]
    round2 = [round1[i]*sum([round1[i+choice1[i % 4]+j]*odds[i][i+choice1[(i % 4)]+j] for j in range(2)]) for i in range(n)]
    round3 = [round2[i]*sum([round2[i+choice2[i % 8]+j]*odds[i][i+choice2[(i % 8)]+j] for j in range(4)]) for i in range(n)]
    round4 = [round3[i]*sum([round3[i+choice3[i % 16]+j]*odds[i][i+choice3[(i % 16)]+j] for j in range(8)]) for i in range(n)]
    return round4[bracket.index(seed)]

bracket = [1,16,8,9,5,12,4,13,6,11,3,14,7,10,2,15]
bracket2= [1,3,8,9,5,12,4,13,6,11,16,14,7,10,2,15]

seed = 2
print("For the original bracket :")
print("Probability of seed {} winning is {:.2f}%".format(seed,float(calc(seed,bracket))*100))
print("\nSwapping 3-16 gives :")
print("Probability of seed {} winning is {:.2f}%".format(seed,float(calc(seed,bracket2))*100))
print("\nDifference is {:.5f}%".format(float(calc(seed,bracket2)-calc(seed,bracket))*100))

For the original bracket :
Probability of seed 2 winning is 21.60%

Swapping 3-16 gives :
Probability of seed 2 winning is 28.16%

Difference is 6.55795%
